In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[4]") \
.appName("SalesPrediction") \
.config("spark.executor.memory", "1g") \
.config("spark.driver.memory", "1g") \
.getOrCreate()

In [ ]:
adv = spark.read.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.option("sep", ",") \
.load("/home/alper/Spark/data/Advertising.csv")

In [ ]:
adv.limit(5).toPandas().head()

In [ ]:
adv.describe().toPandas()

In [ ]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler() \
.setInputCols(["TV", "Radio", "Newspaper"]) \
.setOutputCol("features")

In [ ]:
from pyspark.ml.regression import LinearRegression

lr_object = LinearRegression() \
.setFeaturesCol("features") \
.setLabelCol("Sales")

In [ ]:
train_df, test_df = adv.randomSplit([0.8, 0.2], seed=4242)

In [ ]:
from pyspark.ml import Pipeline

pipe = Pipeline() \
.setStages([vector_assembler, lr_object])

In [ ]:
model = pipe.fit(train_df)

In [ ]:
resultDF = model.transform(train_df)

In [ ]:
resultDF.limit(5).toPandas().head()

In [ ]:
model.stages

In [ ]:
lr_model = model.stages[1]

In [ ]:
lr_model.coefficients

In [ ]:
lr_model.intercept

In [ ]:
print("y =", lr_model.intercept, "(TV *", lr_model.coefficients[0],
      ") + (Radio *", lr_model.coefficients[1], ") + (Newspaper *", lr_model.coefficients[2], ")")

In [ ]:
print("R2 Score: ", lr_model.summary.r2)

In [ ]:
print("RMSE Score: ", lr_model.summary.rootMeanSquaredError)

In [ ]:
print("MAE Score: ", lr_model.summary.meanAbsoluteError)

In [ ]:
print("MSE Score: ", lr_model.summary.meanSquaredError)

In [ ]:
import pandas as pd

test = {"TV": [10.0], "Radio": [15.0], "Newspaper": [20.0]}
testDF = pd.DataFrame(test)
testDF.head()

In [ ]:
predictDF = spark.createDataFrame(testDF)
predictDF.show()

In [ ]:
predictDF = vector_assembler.transform(predictDF)

In [ ]:
lr_model.transform(predictDF).toPandas().head()